In [33]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings

In [2]:
def get_table_names(host, user, password, database):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query to retrieve table names
        query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
        cursor.execute(query)

        # Fetch all the rows
        tables = cursor.fetchall()

        # Print table names
        # for (table_name,) in tables:
        #     print(table_name)

    except mysql.connector.Error as error:
        print(f"Error: {error}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            return tables

In [3]:
def table_to_dataframe(host, user, password, database, table_name):
    try:
        # Establish a database connection
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Use backticks around the table name in the query
        query = f"SELECT * FROM `{table_name}`"

        # Use pandas to read the query into a DataFrame
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            df = pd.read_sql(query, conn)

        return df

    except mysql.connector.Error as error:
        print(f"Error: {error}")
        return None
    finally:
        if conn.is_connected():
            conn.close()

In [4]:

# databaseA = "A" + datetime.now().strftime("%Y%m%d") 
databaseA = 'a20240111'
password = '93150lbm!!'

tables = get_table_names('127.0.0.1', 'root', password, databaseA)

print(tables)

[('009520',), ('009620',), ('009730',), ('010170',), ('013990',), ('016790',), ('021080',), ('024060',), ('025320',), ('025980',), ('027360',), ('027830',), ('028300',), ('032500',), ('032580',), ('033540',), ('036090',), ('036540',), ('036630',), ('038870',), ('039560',), ('039980',), ('041020',), ('041190',), ('041460',), ('042510',), ('047310',), ('049080',), ('050890',), ('054920',), ('056360',), ('056700',), ('057680',), ('058110',), ('060230',), ('060250',), ('060280',), ('060310',), ('064260',), ('065350',), ('065450',), ('065530',), ('066790',), ('072950',), ('080220',), ('080530',), ('086960',), ('088800',), ('090150',), ('094170',), ('094480',), ('096630',), ('099440',), ('100590',), ('101730',), ('112040',), ('115180',), ('124500',), ('129890',), ('131370',), ('136480',), ('142210',), ('153490',), ('200470',), ('203650',), ('205100',), ('207760',), ('214270',), ('214680',), ('232140',), ('235980',), ('242040',), ('250060',), ('253590',), ('264850',), ('270660',), ('289220',)

In [5]:
signal = []
for (code,) in tables : 

    df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)

    if df is not None:
        df['체결시간'] = pd.to_datetime(df['체결시간'])
        df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
        df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
        df['VolumePrice'] = df['현재가'] * df['거래량']

        threshold = 100000000
        money_sum = df['VolumePrice'][df['체결시간'] > '2024-01-11 15:19:30'].sum()
        hey = money_sum  > threshold
        if hey : 
            
            signal.append({code:money_sum})
            print({code:money_sum})


print(signal)
        
        

{'009520': 113611600}
{'027830': 332783790}


UnboundLocalError: local variable 'conn' referenced before assignment

In [6]:
import pandas as pd
import mplfinance as mpf
code  = tables[0][0]
print(code)


df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)

# print(df)
df['체결시간'] = pd.to_datetime(df['체결시간'])
df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
df['거래량'] = pd.to_numeric(df['거래량'].str.replace('+','').str.replace('-',''), errors='coerce')

significant_intervals = pd.DataFrame(columns=['Start_Time', 'End_Time', 'Min_Price', 'Max_Price','Fluctuation'])
for i in range(len(df)):
    # Define the start time of the interval
    start_time = df.loc[i, '체결시간']

    # Find the index where the time exceeds start time by 1 minute
    end_index = df[df['체결시간'] > start_time + pd.Timedelta(minutes=2)].index.min()

    # If end_index is not NaN, calculate min and max prices in the interval
    if pd.notna(end_index):
        end_time = df.loc[end_index, '체결시간']
        interval_data = df.loc[i:end_index]
        min_price = interval_data['현재가'].min()
        max_price = interval_data['현재가'].max()
        fluctuation = ((max_price - min_price) /min_price ) * 100
        # new_df = pd.DataFrame({
        #     'Start_Time': start_time,
        #     'End_Time': end_time,
        #     'Min_Price': min_price,
        #     'Max_Price': max_price
        # })
        new_row = {
            'Start_Time': start_time,
            'End_Time': end_time,
            'Min_Price': min_price,
            'Max_Price': max_price,
            'Fluctuation' : fluctuation
        }
        new_row_df = pd.DataFrame([new_row])

        significant_intervals = pd.concat([significant_intervals, new_row_df], ignore_index=True)

# Print the significant intervals DataFrame
print(significant_intervals)

009520


C:\Users\pari0\AppData\Local\Temp\ipykernel_11004\421468811.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  significant_intervals = pd.concat([significant_intervals, new_row_df], ignore_index=True)


                     Start_Time                   End_Time Min_Price  \
0    2024-01-11 13:09:56.486149 2024-01-11 13:11:58.059445     29800   
1    2024-01-11 13:09:59.850294 2024-01-11 13:11:59.858734     29800   
2    2024-01-11 13:10:04.490936 2024-01-11 13:12:07.853627     29800   
3    2024-01-11 13:10:07.053451 2024-01-11 13:12:07.853627     29800   
4    2024-01-11 13:10:24.954138 2024-01-11 13:12:30.558711     29800   
...                         ...                        ...       ...   
2555 2024-01-11 15:17:42.173461 2024-01-11 15:19:49.580640     29400   
2556 2024-01-11 15:17:42.210488 2024-01-11 15:19:49.580640     29400   
2557 2024-01-11 15:17:48.481036 2024-01-11 15:19:49.580640     29400   
2558 2024-01-11 15:17:48.622905 2024-01-11 15:19:49.580640     29400   
2559 2024-01-11 15:17:55.607265 2024-01-11 15:19:55.885320     29400   

     Max_Price  Fluctuation  
0        29850     0.167785  
1        29850     0.167785  
2        29850     0.167785  
3        29850 

In [12]:
import pandas as pd
import mplfinance as mpf
from tqdm.notebook import tqdm
signal = {}

for (code,) in tqdm(tables) : 
    df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)

    # print(df)
    df['체결시간'] = pd.to_datetime(df['체결시간'])
    df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
    df['거래량'] = pd.to_numeric(df['거래량'].str.replace('+','').str.replace('-',''), errors='coerce')

    significant_intervals = pd.DataFrame(columns=['Start_Time', 'End_Time', 'Min_Price', 'Max_Price','Fluctuation'])
    for i in range(len(df)):
        # Define the start time of the interval
        start_time = df.loc[i, '체결시간']

        # Find the index where the time exceeds start time by 1 minute
        end_index = df[df['체결시간'] > start_time + pd.Timedelta(minutes=2)].index.min()

        # If end_index is not NaN, calculate min and max prices in the interval
        if pd.notna(end_index):
            end_time = df.loc[end_index, '체결시간']
            interval_data = df.loc[i:end_index]
            min_price = interval_data['현재가'].min()
            max_price = interval_data['현재가'].max()
            fluctuation = ((max_price - min_price) /min_price ) * 100
            
            start_price =  interval_data['현재가'].iloc[0]
            end_price = interval_data['현재가'].iloc[-1]
            
            if fluctuation > 3 and end_price > start_price: 
                print(code, start_time)
                if code not in signal.keys():
                    signal[code] = []
                signal[code].append(start_time)

    

  0%|          | 0/100 [00:00<?, ?it/s]

021080 2024-01-11 13:32:16.728708
021080 2024-01-11 13:32:17.147220
027830 2024-01-11 14:07:22.375476
027830 2024-01-11 14:07:22.786947
027830 2024-01-11 14:07:22.887725
027830 2024-01-11 14:07:22.980638
027830 2024-01-11 14:07:23.979931
027830 2024-01-11 14:07:24.573713
027830 2024-01-11 14:07:29.307055
027830 2024-01-11 14:07:29.581620
027830 2024-01-11 14:07:29.779629
027830 2024-01-11 14:07:32.904973
027830 2024-01-11 14:07:33.783438
027830 2024-01-11 14:07:44.379600
027830 2024-01-11 14:07:35.974254
027830 2024-01-11 14:07:36.170859
027830 2024-01-11 14:07:37.703469
027830 2024-01-11 14:07:38.913581
027830 2024-01-11 14:07:46.379019
027830 2024-01-11 14:07:46.770590
027830 2024-01-11 14:07:40.377449
027830 2024-01-11 14:07:41.293225
027830 2024-01-11 14:07:50.083587
027830 2024-01-11 14:07:57.291519
027830 2024-01-11 14:07:57.572025
027830 2024-01-11 14:07:51.571538
027830 2024-01-11 14:08:03.370377
027830 2024-01-11 14:07:59.778240
027830 2024-01-11 14:08:01.372378
027830 2024-01

In [32]:
import numpy as np 

for (code,) in tqdm(tables): 
    df = table_to_dataframe('127.0.0.1', 'root', password, databaseA, code)
    
    df['체결시간'] = pd.to_datetime(df['체결시간'])
    df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
    df['전체거래량'] = pd.to_numeric(df['거래량'].str.replace('+','').str.replace('-',''), errors='coerce')
    df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
    
    df['매수'] = df['거래량'].apply(lambda x: x if x > 0 else 0)
    df['매도'] = df['거래량'].apply(lambda x: abs(x) if x < 0 else 0)
    
    df['거래대금'] = df['현재가'] * df['거래량']
    
    significant_intervals = pd.DataFrame(columns=['Start_Time', 'End_Time', 'Min_Price', 'Max_Price', 'Fluctuation', 'Avg_Volume'])
    
    start_time_standard = df.loc[0, '체결시간'] + pd.Timedelta(minutes=5)
    for i in range(len(df)):
        start_time = df.loc[i, '체결시간']
        
        if start_time < start_time_standard :
            continue
        else : 
            # Find the start time of the 5-minute interval preceding the current time
            interval_start_time = start_time - pd.Timedelta(minutes=5)

            time1 = interval_start_time
            time2 = time1 + pd.Timedelta(minutes=1)
            time3 = time2 + pd.Timedelta(minutes=1)
            time4 = time3 + pd.Timedelta(minutes=1)
            time5 = time4 + pd.Timedelta(minutes=1)
            time6 = time5 + pd.Timedelta(minutes=1)
            
            
            real_time_interval = start_time + pd.Timedelta(seconds=15)
            
            
            
            
            
            
            inter1 = df['전체거래량'][(df['체결시간'] >= time1) & (df['체결시간'] < time2)].sum()
            inter2 = df['전체거래량'][(df['체결시간'] >= time2) & (df['체결시간'] < time3)].sum()
            inter3 = df['전체거래량'][(df['체결시간'] >= time3) & (df['체결시간'] < time4)].sum()
            inter4 = df['전체거래량'][(df['체결시간'] >= time4) & (df['체결시간'] < time5)].sum()
            inter5 = df['전체거래량'][(df['체결시간'] >= time5) & (df['체결시간'] < time6)].sum()
            
            inter6 = df['전체거래량'][(df['체결시간'] >= time6) & (df['체결시간'] < real_time_interval)].sum()
            inter6_buy = df['매수'][(df['체결시간'] >= time6) & (df['체결시간'] < real_time_interval)].sum()
            inter6_sell = df['매도'][(df['체결시간'] >= time6) & (df['체결시간'] < real_time_interval)].sum()
            inter6_amount = df['거래대금'][(df['체결시간'] >= time6) & (df['체결시간'] < real_time_interval)].sum()
            
            intervals_sum = [inter1, inter2, inter3, inter4, inter5]

            inter_mean = np.mean(intervals_sum)
            

            if inter6 > inter_mean * 5 and inter6_buy > inter6_sell *5 and inter6_amount > 100000000: 
                print(inter6)
                print(inter_mean)
                print(inter6_buy)
                print(inter6_sell)
                print(f'signal - code : {code} , time : {real_time_interval}')
                input()

            
            


  0%|          | 0/100 [00:00<?, ?it/s]

7058
1042.8
7008
50
signal - code : 009520 , time : 2024-01-11 14:47:12.221850
7267
1056.2
7007
260
signal - code : 009520 , time : 2024-01-11 14:47:06.121179
7068
1096.0
7007
61
signal - code : 009520 , time : 2024-01-11 14:47:07.229959
7068
1040.8
7008
60
signal - code : 009520 , time : 2024-01-11 14:47:10.120551
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
7108
1025.8
7048
60
signal - code : 009520 , time : 2024-01-11 14:47:17.917525
31467
6113.8
29410
2057
signal - code : 013990 , ti

KeyboardInterrupt: Interrupted by user